In [16]:
import pandas as pd
import pickle
import sys
import os

# Get the notebook's directory and add its parent directory to sys.path
notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(parent_dir)
from constants import *

In [17]:
eco_mapping_df = pd.read_csv("../data/ECO_codes_mapping.csv")
eco_mapping = {eco: idx for idx, eco in enumerate(eco_mapping_df["eco"].unique())}

In [18]:
len(eco_mapping.keys())

496

In [20]:
def load_preprocessed_data(filename):
    with open(filename, 'rb') as f:
        df = pickle.load(f)
    return df
df = load_preprocessed_data(os.path.join('../', PREPROCESSED_DIR, "lichess_games_20K.pkl"))

In [21]:
df.describe

<bound method NDFrame.describe of                                                 matrices opening_eco  \
0      [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         D10   
1      [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         B00   
2      [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         C20   
3      [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         D02   
4      [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         C41   
...                                                  ...         ...   
20050  [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         A80   
20051  [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         A41   
20052  [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         D00   
20053  [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         B07   
20054  [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         D00   

       opening_ply  
0                5  
1                4  
2                3  
3                

In [22]:
df["opening_eco"].value_counts()

opening_eco
A00    1007
C00     844
D00     739
B01     716
C41     691
       ... 
D58       1
C32       1
E18       1
D95       1
D19       1
Name: count, Length: 364, dtype: int64